<a href="https://colab.research.google.com/github/ShaiShmuel/fake_news_detection/blob/main/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fake news detaction by title | Uriel Bender & Shai Shmuel

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

Reading the Data set

In [ ]:
Train_data = pd.read_csv("/content/fake-news/train.csv")
Train = Train_data.copy()

In [ ]:
# here we are printing first 15 lines of our train dataset
Train[0:15]

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [ ]:
Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [ ]:
Train.shape

(20800, 5)

In [ ]:
# here we are checking if there is null values
Train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
Train = Train.dropna()

In [ ]:
Train.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
Train[0:10]

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


We can see that we have missing values in our training set, id 6 and 8 are missing

In [ ]:
news_type = pd.crosstab(Train.label, Train.title)
news_type

title,"""Allahu Akbar, the Russians are here!"": Aleppo terrorists flee Russian airstrike (VIDEO)","""Authoritarianism"": How the West demonizes strong, popular leaders","""Donald, willst du mich heiraten?"": Clinton hat Kampf ums Weiße Haus noch nicht aufgegeben","""Europe Has No Future"" Putin Slams Austria Migrant Child Rape Injustice","""Fake News"" About Trump Continues Unabated","""France First"" – Le Pen Hits Out At Islamism and Globalisation","""Hamilton"" Actor Refuses To Apologize For Political Stunt","""Just say it's the Russians"" - Joaquin Flores on RT [Video]","""Top Five Clinton Donors Are Jewish"" - How Anti-Semitic Is This Fact?","""Zu unseriös"": Auch Lothar Matthäus sagt HSV als Sportdirektor ab",#2816: Clinton Pride’s 8(a) Pig Farm Bridge – Serco Zulu Server Snuff –Soros's Patented Voter Key,#2817: Serco's Zulu Starnet Blackmail – Clinton 8(a) Pig Farm Bridge – Soros Patent Vote,#2820: Clinton 8(a) Pig Farm Servers – Boeing Bridge Of Pimps – Serco In-Flight Zulu Snuff,#BREAKING: SECOND Assassination Attempt On Trump In NV; Suspect Detained (LIVE BLOG),#BREAKING: TRUMP USHERED OFF STAGE IN NV SUSPECT DETAINED,#BoycottComedian…ROBERT DENIRO Wanted “To Punch Trump In The Face”…Supports Anti-Trump Rioters…Now Wants Americans To Support His New Movie [VIDEO],#DraftOurDaughters: Feminist Hillary Supporters Vow To Fight War With Russia For Us,#DraftOurDaughters: Pro-War Hillary Faces Backlash Over Female Draft,#GrammysSoWhite Came to Life. Will the Awards Face Its Race Problem? - The New York Times,#InNorthDakota ~~ PALESTINIANS STAND WITH THE SIOUX,#MayorsStand4All Day Touts Support for Illegal Immigrants,#MyUnintendedJoy: Americans Show Why They Choose Life Over Abortion - Breitbart,#NoDAPL Situation Critical,#NoDAPL Spills Over: Musicians Boycott Dakota Access Pipeline CEO’s Record Label & Festival,#NoDAPL: Angry Man Fires 7 Shots Into Air Near Standing Rock Water Protectors,#Podesta21: Clinton’s ‘Twisted’ Behavior ‘Smacks of Acting Above the Law’,#PodestaEmails20: WikiLeaks releases another batch from Clinton campaign chair,#ProtestPP: Nationwide Protests Against Government Funding of Planned Parenthood Underway - Breitbart,#RESIST as Marketing: ’Fearless Girl’ Statue Taking Home Awards at Cannes - Breitbart,#TrumpProtest: Communists Mobilize To Disrupt President-Elect Trump’s Inauguration,"$2,700 for Hillary Clinton at ‘Hamilton’? That Would Be Enough - The New York Times",$20 Billion in Tax Credits Fails to Increase College Attendance - The New York Times,$25 Million in Breakthrough Prizes Given in Science and Math - The New York Times,"$50,000 Reward for Facebook Killer as Manhunt Goes ’National’ - Breitbart",$6 Million Later: North Dakota Police Can’t Afford The Pipeline Protest,$7 Million in Donations to Go Directly to Orlando Kin and Survivors - The New York Times,$70 Million Awarded to Woman with Ovarian Cancer Due to Baby Powder Usage,'All About Jesus' shirt sparks school uproar,'Alt-Right' Meeting Full Video Replay: People Give Hitler Salute For Donald Trump,'D**k-Waving Berlusconi Knockoff': Late-Night Host's Nasty Trump Attack Dismantled by Steven Crowder,...,"“Man hat Angst, das Haus zu verlassen”",“Memoirs Of A Misogynist” Is A Brutal And Entertaining Look At Our Sexual Dystopia,"“Mommy am I Going To Die?” Mom Calls 911 For Paramedic, Cop Shows Up, Shoots 4 Year Old Daughter",“Mr. Wonderful te hace pensar a saco”,“My Trampoline Addiction Hell” Buster The Boxer Tells All,"“National Mood” Focus Group Reflects Angry, Divided America",“Not A ‘Real’ Union”: Emails Show Clinton Campaign Attacks On Nurses,“Nothing Good Can Come of This Election”–and That’s Good,“Organic” Food From China Found To Be Highly Contaminated,“People Power” Versus Manipulation of the Masses,“Regime Change”? South Africa Targeted by Western Destabilization Efforts? | Global Research,“Rigged To Deny Trump”: Are They Stealing These States to Block Electoral Path to 270?,“Rigged To Deny Trump”: Looks Like Elite Trying to Steal These

In [ ]:
Train.groupby('label').count()

,id,title,author,text
label,,,,
0,10361,10361,10361,10361
1,7924,7924,7924,7924


We want to check if all of our data have a reasonable length

In [ ]:
length = []
[length.append(len(str(text))) for text in Train['title']]
Train['title-length'] = length
Train.head()

,id,title,author,text,label,title-length
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,81
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,55
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,33
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,63
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,93


In [ ]:
min(Train['title-length']), max(Train['title-length']), round(sum(Train['title-length'])/len(Train['title-length']))

(3, 456, 75)

In [ ]:
def get_outliers(df, series):
  q1 = series.quantile(0.25)
  q3 = series.quantile(0.75)

  if q1*q3 == 0:
    iqr = abs(2*(q1+q3))
    toprange = iqr
    botrange = -toprange
  else:
    iqr = q3-q1
    toprange = q3 + iqr * 1.5
    botrange = q1 - iqr * 1.5

  outliers_top=df[series > toprange]
  outliers_bot= df[series < botrange]
  outliers = pd.concat([outliers_bot, outliers_top], axis=0)

  return (botrange, toprange, outliers)

In [ ]:
botrange, toprange, outliers = get_outliers(Train, Train['title-length'])
print(toprange)
print(botrange)
outliers

128.5
20.5


,id,title,author,text,label,title-length
86,86,Poison,Heather Callaghan,By Dr. Mark Sircus Everyone knows that there a...,1,6
156,156,The Battle at UNESCO,Ari Lieberman,"The Muslim Bloc may have won the battle, but d...",1,20
248,248,The Left’s Vision,Thomas Sowell,"By Thomas Sowell November 1, 2016 \nThe politi...",1,17
257,257,Newsticker (974),noreply@blogger.com (Der Postillon),+++ Hände weg vom Alkohol: Schreiner rührt nac...,1,16
347,347,LesserOfTwoEvilism,Anonymous,2016 presidential campaign by Matt Sedillo \nH...,1,18
...,...,...,...,...,...,...
20223,20223,Comment on Understanding Misleading Food Label...,Understanding Misleading Food Labels: An Infog...,Share on Facebook Share on Twitter “Let food b...,1,277
20277,20277,"Like a good little sharia-compliant female, Pr...",BareNakedIslam,BNI Store Nov 6 2016 Like a good little sharia...,1,165
20324,20324,THE END GAME closes in on the Clintons as the ...,Dave Hodges,THE END GAME closes in on the Clintons as the ...,1,188
20513,20513,SAID IN SPANISH: A Mexican Governor Meddles In...,Allan Wall,,1,156


In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(name='Title length', y=Train['title-length'], boxmean='sd'))

fig.update_traces(
    marker=dict(
        color='rgb(8,81,156)',
        outliercolor='rgba(219, 64, 82, 0.6)',
        line=dict(
            outliercolor='rgba(219, 64, 82, 0.6)',
            outlierwidth=2)),
    line_color='rgb(8,81,156)'
)
fig.show()

In [ ]:
Train.shape

(18285, 6)

In [ ]:
len(Train[Train['title-length'] < 20])


158

In [ ]:
len(Train[Train['title-length'] > 128])


214

We would want to drop all of our outliers so we can get more accurate result

In [ ]:
'''for title in Train['title-length']:
  if(title > 128):
   title = 0'''
  

"for title in Train['title-length']:\n  if(title > 128):\n   title = 0"

In [ ]:
Train.shape

(18285, 6)

In [ ]:
'''# Get indexes where name column doesn't have value john
length_0 = Train[(Train['title-length'] == 0)].index 
# Delete these row indexes from dataFrame
Train.drop(length_0 , inplace=True)'''

"# Get indexes where name column doesn't have value john\nlength_0 = Train[(Train['title-length'] == 0)].index \n# Delete these row indexes from dataFrame\nTrain.drop(length_0 , inplace=True)"

In [ ]:
Train.drop(Train[Train['title-length'] > 128].index, inplace = True)
Train.drop(Train[Train['title-length'] < 20].index, inplace = True)

In [ ]:
Train.shape

(17913, 6)